### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [2]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [3]:
X_test.head(3)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,70.0,0,582,0,40,0,51000.0,2.7,136,1,1,250,0
1,50.0,1,298,0,35,0,362000.0,0.9,140,1,1,240,0
2,45.0,0,2442,1,30,0,334000.0,1.1,139,1,0,129,1


In [7]:
import urllib.request
import json      


def get_prediction(x):
    age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction, high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex, smoking, time = x
    body = {'age': age,
                            'anaemia': anaemia,
                            'creatinine_phosphokinase': creatinine_phosphokinase,
                            'diabetes': diabetes,
                            'ejection_fraction': ejection_fraction,
                            'high_blood_pressure': high_blood_pressure,
                            'platelets': platelets,
                            'serum_creatinine': serum_creatinine,
                            'serum_sodium': serum_sodium,
                            'sex': sex,
                            'smoking': smoking,
                            'time': time}

    myurl = "http://127.0.0.1:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']


In [8]:
%%time
predictions = X_test[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 
                        'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 
                        'smoking', 'time']].iloc[:500].apply(lambda x: get_prediction(x), 1)

URLError: ignored

In [ ]:
predictions

0      0.026174
1      0.044576
2      0.004804
3      0.001057
4      0.001229
         ...   
495    0.008572
496    0.001839
497    0.620032
498    0.001418
499    0.001099
Length: 500, dtype: float64

In [ ]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525